In [1]:
!wget https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py

--2022-04-27 23:36:21--  https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1654 (1.6K) [text/plain]
Saving to: 'average_precision.py.1'

average_precision.p 100%[===================>]   1.62K  --.-KB/s    in 0s      

2022-04-27 23:36:21 (17.0 MB/s) - 'average_precision.py.1' saved [1654/1654]



In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from average_precision import apk

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)

from average_precision import apk

def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []
    no_purchases_pattern = []
    
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [2]:
import pandas as pd

In [67]:
%%time

transactions = pd.read_parquet('transactions_train.parquet')
customers = pd.read_parquet('customers.parquet')
articles = pd.read_parquet('articles.parquet')

CPU times: user 3.15 s, sys: 2.06 s, total: 5.21 s
Wall time: 1.69 s


In [68]:
VALID = True

if VALID:
    transactions = transactions[transactions.t_dat<'2020-09-16']

In [69]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [70]:
%%time
c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 31.3 s, sys: 1.68 s, total: 32.9 s
Wall time: 31 s


In [71]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 805 ms, sys: 86.7 ms, total: 891 ms
Wall time: 888 ms


In [72]:
mean_price = transactions.groupby(['week', 'article_id'])['price'].mean()

In [73]:
# bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()

bestsellers_previous_week = pd.read_parquet('valid_best_sell_all.parquet')
bestsellers_previous_week = pd.merge(bestsellers_previous_week, 
                                     mean_price, on=['week', 'article_id']).reset_index(drop=True)
bestsellers_previous_week.week += 1

In [74]:
bestsellers_previous_week

,week,article_id,bestseller_rank,RankType,price
0,95,806388001,1,MostBought,0.013301
1,95,806388003,1,SecondMost,0.013223
2,95,806388002,1,LowerPrice,0.013238
3,95,806388017,1,AnotherColor,0.013284
4,95,806388009,1,AnotherAppearance,0.013236
...,...,...,...,...,...
416,104,762846008,9,LowerPrice,0.025013
417,104,762846006,9,AnotherColor,0.025155
418,104,673677002,11,MostBought,0.024925
419,104,673677022,11,SecondMost,0.025012


In [75]:
transactions.shape

(2809233, 6)

In [76]:
transactions = transactions.merge(bestsellers_previous_week[['article_id', 'RankType', 'week']], 
                   on=['article_id','week'], how='left')

In [77]:
transactions.shape

(2826521, 7)

In [78]:
transactions['RankType'].value_counts()

MostBought           37569
SecondMost           25172
LowerPrice           11599
AnotherColor          6168
AnotherAppearance     1679
Name: RankType, dtype: int64

In [79]:
transactions[transactions.customer_id==28847241659200]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType
247958,2020-07-14,28847241659200,730683036,0.042356,1,94,NaN
247959,2020-07-14,28847241659200,851094001,0.011000,1,94,NaN
247960,2020-07-14,28847241659200,757303012,0.030492,1,94,NaN
426786,2020-07-18,28847241659200,762846001,0.025407,1,95,NaN
426787,2020-07-18,28847241659200,829308001,0.033881,1,95,NaN
756652,2020-07-26,28847241659200,887770001,0.016932,1,96,NaN
2223356,2020-08-31,28847241659200,760084003,0.025407,1,101,NaN
2334304,2020-09-03,28847241659200,925246001,0.128797,2,102,NaN


In [80]:
candidates_last_purchase = transactions.copy()

In [81]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 6.96 s, sys: 55.3 ms, total: 7.02 s
Wall time: 7.01 s


In [82]:
candidates_last_purchase[candidates_last_purchase.customer_id==28847241659200]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType
247958,2020-07-14,28847241659200,730683036,0.042356,1,95,NaN
247959,2020-07-14,28847241659200,851094001,0.011000,1,95,NaN
247960,2020-07-14,28847241659200,757303012,0.030492,1,95,NaN
426786,2020-07-18,28847241659200,762846001,0.025407,1,96,NaN
426787,2020-07-18,28847241659200,829308001,0.033881,1,96,NaN
756652,2020-07-26,28847241659200,887770001,0.016932,1,101,NaN
2223356,2020-08-31,28847241659200,760084003,0.025407,1,102,NaN
2334304,2020-09-03,28847241659200,925246001,0.128797,2,104,NaN


In [83]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price', 'RankType']) \
    .copy()

In [84]:
unique_transactions[unique_transactions.week==94]

,t_dat,customer_id,sales_channel_id,week
0,2020-07-08,857913002275398,1,94
3,2020-07-08,1658289241058394,1,94
4,2020-07-08,3828854365940846,1,94
6,2020-07-08,4195624216542755,1,94
11,2020-07-08,4233235614030232,2,94
...,...,...,...,...
286620,2020-07-14,18436753767023068280,2,94
286628,2020-07-14,18438250417436054613,2,94
286636,2020-07-14,18440748117155410745,1,94
286641,2020-07-14,18444695778382839691,1,94


In [85]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [86]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,bestseller_rank,RankType,price
0,2020-07-15,272412481300040,1,95,806388001,1,MostBought,0.013301
1,2020-07-15,272412481300040,1,95,806388003,1,SecondMost,0.013223
2,2020-07-15,272412481300040,1,95,806388002,1,LowerPrice,0.013238
3,2020-07-15,272412481300040,1,95,806388017,1,AnotherColor,0.013284
4,2020-07-15,272412481300040,1,95,806388009,1,AnotherAppearance,0.013236
...,...,...,...,...,...,...,...,...
29863840,2020-09-15,18446630855572834764,2,103,909916001,11,MostBought,0.033403
29863841,2020-09-15,18446630855572834764,2,103,762846031,12,MostBought,0.024715
29863842,2020-09-15,18446630855572834764,2,103,762846026,12,SecondMost,0.024913
29863843,2020-09-15,18446630855572834764,2,103,762846027,12,LowerPrice,0.024895


In [87]:
candidates_bestsellers['RankType'].value_counts()

MostBought           8240712
SecondMost           7725138
LowerPrice           6035043
AnotherColor         4882148
AnotherAppearance    2980804
Name: RankType, dtype: int64

In [88]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [89]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [90]:
candidates_bestsellers_test_week

,t_dat,customer_id,sales_channel_id,week,article_id,bestseller_rank,RankType,price
0,2020-07-08,857913002275398,1,104,909370001,1,MostBought,0.032947
1,2020-07-08,857913002275398,1,104,865799006,2,MostBought,0.033340
2,2020-07-08,857913002275398,1,104,865799005,2,SecondMost,0.033233
3,2020-07-08,857913002275398,1,104,918522001,3,MostBought,0.041416
4,2020-07-08,857913002275398,1,104,924243001,4,MostBought,0.041549
...,...,...,...,...,...,...,...,...
13181035,2020-09-15,18446630855572834764,2,104,762846008,9,LowerPrice,0.025013
13181036,2020-09-15,18446630855572834764,2,104,762846006,9,AnotherColor,0.025155
13181037,2020-09-15,18446630855572834764,2,104,673677002,11,MostBought,0.024925
13181038,2020-09-15,18446630855572834764,2,104,673677022,11,SecondMost,0.025012


In [91]:
candidates_bestsellers_test_week['RankType'].value_counts()

MostBought           5272416
SecondMost           3954312
LowerPrice           1757472
AnotherColor         1757472
AnotherAppearance     439368
Name: RankType, dtype: int64

In [92]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

candidates_bestsellers = candidates_bestsellers[candidates_last_purchase.columns.tolist()]

# Combining transactions and candidates / negative examples

In [93]:
transactions['purchased'] = 1

In [94]:
transactions[transactions.customer_id==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType,purchased
286672,2020-07-15,272412481300040,778064028,0.008458,1,95,MostBought,1
286673,2020-07-15,272412481300040,816592008,0.016932,1,95,NaN,1
286674,2020-07-15,272412481300040,621381021,0.033881,1,95,NaN,1
286675,2020-07-15,272412481300040,817477003,0.025407,1,95,NaN,1
286676,2020-07-15,272412481300040,899088002,0.025407,1,95,NaN,1
577387,2020-07-22,272412481300040,885077001,0.008458,1,96,NaN,1
668994,2020-07-24,272412481300040,850176003,0.029034,2,96,NaN,1
668995,2020-07-24,272412481300040,875803001,0.064559,2,96,NaN,1
668996,2020-07-24,272412481300040,892970003,0.020966,2,96,NaN,1
668997,2020-07-24,272412481300040,854619003,0.020966,2,96,NaN,1


In [95]:
candidates_last_purchase[candidates_last_purchase.customer_id==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType
286672,2020-07-15,272412481300040,778064028,0.008458,1,96,MostBought
286673,2020-07-15,272412481300040,816592008,0.016932,1,96,NaN
286674,2020-07-15,272412481300040,621381021,0.033881,1,96,NaN
286675,2020-07-15,272412481300040,817477003,0.025407,1,96,NaN
286676,2020-07-15,272412481300040,899088002,0.025407,1,96,NaN
577387,2020-07-22,272412481300040,885077001,0.008458,1,103,NaN
668994,2020-07-24,272412481300040,850176003,0.029034,2,103,NaN
668995,2020-07-24,272412481300040,875803001,0.064559,2,103,NaN
668996,2020-07-24,272412481300040,892970003,0.020966,2,103,NaN
668997,2020-07-24,272412481300040,854619003,0.020966,2,103,NaN


In [99]:
candidates_bestsellers[candidates_bestsellers.customer_id==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType
0,2020-07-15,272412481300040,806388001,0.013301,1,95,MostBought
1,2020-07-15,272412481300040,806388003,0.013223,1,95,SecondMost
2,2020-07-15,272412481300040,806388002,0.013238,1,95,LowerPrice
3,2020-07-15,272412481300040,806388017,0.013284,1,95,AnotherColor
4,2020-07-15,272412481300040,806388009,0.013236,1,95,AnotherAppearance
...,...,...,...,...,...,...,...
2279905,2020-07-15,272412481300040,762846008,0.025013,1,104,LowerPrice
2279906,2020-07-15,272412481300040,762846006,0.025155,1,104,AnotherColor
2279907,2020-07-15,272412481300040,673677002,0.024925,1,104,MostBought
2279908,2020-07-15,272412481300040,673677022,0.025012,1,104,SecondMost


In [100]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [101]:
data[data.customer_id==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType,purchased
286672,2020-07-15,272412481300040,778064028,0.008458,1,95,MostBought,1.0
286673,2020-07-15,272412481300040,816592008,0.016932,1,95,NaN,1.0
286674,2020-07-15,272412481300040,621381021,0.033881,1,95,NaN,1.0
286675,2020-07-15,272412481300040,817477003,0.025407,1,95,NaN,1.0
286676,2020-07-15,272412481300040,899088002,0.025407,1,95,NaN,1.0
...,...,...,...,...,...,...,...,...
2279905,2020-07-15,272412481300040,762846008,0.025013,1,104,LowerPrice,0.0
2279906,2020-07-15,272412481300040,762846006,0.025155,1,104,AnotherColor,0.0
2279907,2020-07-15,272412481300040,673677002,0.024925,1,104,MostBought,0.0
2279908,2020-07-15,272412481300040,673677022,0.025012,1,104,SecondMost,0.0


In [102]:
data.shape

(48697927, 8)

In [103]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [104]:
data.shape

(42221170, 8)

In [105]:
data.purchased.mean()

0.059917856373947004

In [106]:
data.groupby('RankType')['purchased'].mean()

RankType
AnotherAppearance    0.000471
AnotherColor         0.000769
LowerPrice           0.001169
MostBought           0.002340
SecondMost           0.001571
Name: purchased, dtype: float64

### Add bestseller Rank

In [107]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [108]:
data[data.customer_id==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType,purchased,bestseller_rank
259636,2020-07-15,272412481300040,778064028,0.008458,1,95,MostBought,1.0,12.0
259637,2020-07-15,272412481300040,816592008,0.016932,1,95,NaN,1.0,NaN
259638,2020-07-15,272412481300040,621381021,0.033881,1,95,NaN,1.0,NaN
259639,2020-07-15,272412481300040,817477003,0.025407,1,95,NaN,1.0,NaN
259640,2020-07-15,272412481300040,899088002,0.025407,1,95,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...
37083219,2020-07-15,272412481300040,762846008,0.025013,1,104,LowerPrice,0.0,9.0
37083220,2020-07-15,272412481300040,762846006,0.025155,1,104,AnotherColor,0.0,9.0
37083221,2020-07-15,272412481300040,673677002,0.024925,1,104,MostBought,0.0,11.0
37083222,2020-07-15,272412481300040,673677022,0.025012,1,104,SecondMost,0.0,11.0


In [109]:
data.week.min()

94

In [111]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)
data.RankType.fillna('Others', inplace=True)

In [112]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [113]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [114]:
data[data.customer_id==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week,RankType,purchased,bestseller_rank,product_code,...,section_name,garment_group_no,garment_group_name,detail_desc,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
106,2020-07-15,272412481300040,778064028,0.008458,1,95,MostBought,1.0,12.0,778064,...,30,1002,2,315,1,1,0,1,48,338849
107,2020-07-15,272412481300040,816592008,0.016932,1,95,Others,1.0,999.0,816592,...,1,1005,0,21685,1,1,0,1,48,338849
108,2020-07-15,272412481300040,621381021,0.033881,1,95,Others,1.0,999.0,621381,...,1,1009,5,415,1,1,0,1,48,338849
109,2020-07-15,272412481300040,817477003,0.025407,1,95,Others,1.0,999.0,817477,...,0,1025,16,5907,1,1,0,1,48,338849
110,2020-07-15,272412481300040,899088002,0.025407,1,95,Others,1.0,999.0,899088,...,1,1012,18,28139,1,1,0,1,48,338849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33148064,2020-07-15,272412481300040,762846008,0.025013,1,104,LowerPrice,0.0,9.0,762846,...,7,1010,6,506,1,1,0,1,48,338849
33148065,2020-07-15,272412481300040,762846006,0.025155,1,104,AnotherColor,0.0,9.0,762846,...,7,1010,6,506,1,1,0,1,48,338849
33148066,2020-07-15,272412481300040,673677002,0.024925,1,104,MostBought,0.0,11.0,673677,...,7,1003,3,577,1,1,0,1,48,338849
33148067,2020-07-15,272412481300040,673677022,0.025012,1,104,SecondMost,0.0,11.0,673677,...,7,1003,3,577,1,1,0,1,48,338849


In [115]:
data.shape

(47704974, 39)

In [130]:
rank_map = {'Others':0, 'MostBought':1, 'SecondMost':2, 'LowerPrice':3, 'AnotherColor':4,'AnotherAppearance':5}
data['RankType'] = data['RankType'].map(rank_map)

In [131]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [117]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

## can use other columns to group with week and cust id - age_bucket for e.g

In [121]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code', 'RankType',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [132]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']
test_X = test[columns_to_use]

CPU times: user 869 ms, sys: 684 ms, total: 1.55 s
Wall time: 1.54 s


# Model training

In [133]:
from lightgbm.sklearn import LGBMRanker

In [156]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type = "dart",
    n_estimators = 3000,
    importance_type='gain',
    verbose=100,
#     num_leaves = 255,
#     max_depth = 8,
)

In [157]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.111164
[LightGBM] [Debug] init for col-wise cost 0.000241 seconds, init for row-wise cost 0.787190 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.241493 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1107
[LightGBM] [Info] Number of data points in the train set: 33147844, number of used features: 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [De

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [D

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debu

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [De

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] 

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM]

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM]

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM]

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
CPU times: user 3d 1h 47min 26s, sys: 3min 10s, total: 3d 1h 50min 36s
Wall time: 1h 53min


In [158]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())
    
# # bestseller_rank 0.9084779803115295
# # article_id 0.028151283853994704
# # product_type_no 0.01703869095478825
# # department_no 0.012842365564634884
# # garment_group_no 0.008488918871462595
# # colour_group_code 0.00508518511811137
# # graphical_appearance_no 0.0043176729897553825
# # section_no 0.004305431560331087
# # perceived_colour_master_id 0.0034837779550319953
# # perceived_colour_value_id 0.0030256368553717135
# # age 0.0018456475582899136
# # index_group_no 0.0011573662182938502
# # index_code 0.0010249484379311765
# # club_member_status 0.0003302095247203395
# # postal_code 0.00031080881628281037
# # fashion_news_frequency 6.535356783770413e-05
# # Active 4.503368971922028e-05
# # FN 3.688151913462962e-06

bestseller_rank 0.6416938607478657
RankType 0.29612175979654076
article_id 0.019123650409472174
product_type_no 0.010144907036869493
department_no 0.00800403808822475
garment_group_no 0.004650143139175998
colour_group_code 0.0044874326096493306
graphical_appearance_no 0.0033976267572211794
section_no 0.002802595582613871
age 0.0024187672454551727
perceived_colour_value_id 0.002253378195498
perceived_colour_master_id 0.0021163953877755513
postal_code 0.0008970514724440002
index_code 0.0007268888740833787
index_group_no 0.0006307651162399336
club_member_status 0.0003381005141626985
Active 9.241826308347583e-05
fashion_news_frequency 8.48440680474669e-05
FN 1.5376695577068447e-05


# Calculate predictions

In [159]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()


CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 19.6 µs


# Create submission

In [160]:
sub = pd.read_csv('sample_submission.csv')

In [161]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 5.39 s, sys: 208 ms, total: 5.6 s
Wall time: 5.59 s


In [162]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds
sub['cust_id'] = customer_hex_id_to_int(sub.customer_id)

In [164]:
valid_data = pd.read_parquet('transactions_train.parquet')
valid_data = valid_data[valid_data.t_dat>='2020-09-16'].copy()

In [165]:
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))

In [166]:
valid_unq = valid_unq.drop('article_id', axis=1).merge(sub[['cust_id','prediction']], left_on='customer_id',
                                          right_on='cust_id', how='inner')

,customer_id,valid_true,cust_id,prediction
0,1402273113592184,0885951001 0611415001,1402273113592184,0909370001 0865799006 0865799005 0918522001 09...
1,1827730561464445,0918603001 0921380001,1827730561464445,0894668003 0715624052 0894668002 0860833018 08...
2,1951136007097426,0778745010,1951136007097426,0909370001 0865799006 0865799005 0918522001 09...
3,2639747769247776,0819547001,2639747769247776,0896161001 0865929003 0889870001 0918522001 09...
4,3177658828628418,0869331006 0866731001,3177658828628418,0909370001 0865799006 0865799005 0918522001 09...
...,...,...,...,...
68979,18444954504588539615,0903062001,18444954504588539615,0807244017 0918292001 0918292004 0751471001 07...
68980,18445164350380731040,0730683050,18445164350380731040,0909370001 0865799006 0865799005 0918522001 09...
68981,18445340048433064259,0714790028,18445340048433064259,0906612002 0751471043 0751471001 0673677002 07...
68982,18445641720816255142,0898713001 0909014001 0919365008 0827635001 08...,18445641720816255142,0909370001 0865799006 0865799005 0918522001 09...


In [167]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # remove this case in advance
    # if not actual:
    #     return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [168]:
mapk(
    valid_unq['valid_true'].map(lambda x: x.split()), 
    valid_unq['prediction'].map(lambda x: x.split()), 
    k=12
)

## 0.02242 with 300 trees
## 0.02238 with 3000 trees

0.02238267000612784